# BERTopic
BERT embeddings과 클래스 기반(class-based) TF-IDF를 활용하여

주제 설명에서 중요한 단어를 유지하면서도 쉽게 해석할 수 있는

조밀한 클러스터를 만드는 토픽 모델링

# 사용 모듈 install and import

## install

In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 2.5 MB/s 
     |████████████████████████████████| 5.2 MB 47.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 662 kB 50.6 MB/s 
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 5.8 MB 47.4 MB/s 
     |████████████████████████████████| 1.3 MB 44.2 MB/s 
     |████████████████████████████████| 182 kB 56.5 MB/s 
     |████████████████████████████████| 7.6 MB 52.8 MB/s 
     |████████████████████████████████| 1.1 MB 43.0 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700820 sha256=797b84ad22308828e98ac297fef4ade9635ff72e8b7f9e62e1c04b0d8be3f5ac
  Stored in directory: /root/.cache/

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 26.55 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 58.6 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-12-02 01:50:28--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (b

## import

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic
import pandas as pd
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# news data topic modeling

## news data load

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/SKKU_ML/data/news_data.csv", encoding='utf-8-sig')

In [ ]:
data_list = ['nv', 'cs', 'cs_sum']
for i in data_list:
    with open(f"/content/drive/MyDrive/SKKU_ML/data/news_{i}.pickle","rb") as fr:
        globals()[f'news_{i}'] = pickle.load(fr)

data dataframe은 크롤링된 news의 date와 content를 가지고 있음

In [ ]:
data

,date,content
0,2021-12-31 16:35:01,사업비 억 원 확보 강소도시로서 경쟁력 끌어올려안동시청안동시가 년 한 해 동안 개의...
1,2021-12-31 15:40:59,의왕시청 전경의왕뉴시스 박석희 기자 경기 의왕시가 심리상담치료 지원에 나선다 의왕...
2,2021-12-31 13:00:01,정동청 서울청정신건강의학과 원장 리스크 회피 불가능 인정해야 해법 발견 가능해정동청...
3,2021-12-31 11:56:23,보건복지부 장관 표창행정안전부 서비스 우수 자치단체 선정괄목 성과 뉴스대전뉴스 백운...
4,2021-12-31 11:43:24,기사내용 요약코로나 블루 극복 위한 웰니스 여행지 자연휴양림이용객 편의 맞춰 규제 ...
...,...,...
2580,2022-10-01 06:06:01,솔루션 저널리즘 현장을 가다 인터뷰 캐서린 롱 시애틀타임스 에듀케이션랩 에디터 철...
2581,2022-09-30 23:13:04,여명 참여 성료인제대학교 대학일자리플러스센터와 주관하고 김해시 김해고용플러스복지...
2582,2022-09-30 17:10:01,서울시정신건강복지센터가 년 정신건강 정책 심포지엄을 개최했다고 밝혔다사진서울시정신건...
2583,2022-09-30 16:08:01,인제대학교총장 전민현 대학일자리플러스센터와 주관하고 김해시 김해고용플러스복지센터...


In [ ]:
docs = []
for i in range(len(data)):
  docs.append(data.content[i])

news_nv data는 mecab() 모듈을 통해 토큰화된 데이터

In [ ]:
news_nv

['사업비 확보 강소 도시 경쟁력 동시 청안 동시 동안 기관 표창 수상 공모 사업 선정 포상금 사업비 확보 경북 북부 중심 도시 경쟁력 다년 지속 코로나 제한 여건 시정 추진 대한민국 일자리 유공 우수 기관 선정 중앙 단위 평가 부문 단위 평가 부문 우수 기관 선정 분야 수상 건강 시정 빈틈없 시정 추진력 인정 행정 분야 민원 행정 추진 실적 평가 최우수 지방 세정 종합 평가 장려상 수상 문화 관광 분야 경상북도 평생 학습 대상 최우 수상 수상 농축 산업 분야 시군 축산 업무 종합 평가 농촌 진흥 사업 종합 평가 최우 수상 수상 환경 도시 분야 경북 환경 평가 최우수 미세먼지 대응 시책 추진 평가 우수 기관 선정 코로나 대두 시민 건강 안전 문제 성과 보건 복지 분야 정신 건강 자살 예방 사업 평가 우수 수상 예산 확보 시정 운영 보탬 공모 사업 선정 사업비 확보 포스트 코로나 시대 대비 각종 정책 원활 추진 중앙 부처 정책 기조 적극 전략 대응 사업 개발 경제 분야 상권 르네상스 사업 전통 시장 시설 현대 사업 선정 사업비 확보 코로나 침체 지역 경제 활성 이바지 교육 분야 지역 특화 평생 교육 활성 지원 사업 사업비 확보 명품 교육 도시 지향 농축 산업 분야 식량 작물 공동 영체 육성 사업 선정 농업 농촌 경쟁력 강화 사업비 확보 문화 관광 분야 제차 문화 도시 지정 세계 유산 축전 확보 작년 관광 거점 도시 지정 문화 거점 도시 발판 마련 환경 도시 분야 취약 지역 생활 여건 개조 사업 경관 디자인 사업 선정 안전 도시 조성 사업비 확보 보건 복지 분야 민관 협혁 국공립 어린이집 지원 사업비 확보 시민 건강 행복 증진 박차 권영세 안동시장 시민 신뢰 응원 직원 헌신 노력 올해 시정 추진 결실 내년 공모 사업 신청 시정 적극 운영 동시 정책 우수 알리 지속 성장 동력 확보',
 '의왕 시청 전경 뉴시스 박석희 기자 경기 의왕시 심리 상담 치료 지원 의왕시 내년 사회 복지 시설 종사자 폭력 피해 상담 치료비 지원 피해 당사자 정신 트라우마 극복 기획 지원 대

## BERTopic - news_nv

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=50,
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(news_nv)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

### visualization

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_distribution(probs[0])

In [ ]:
for i in range(0, 10):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
1 번째 토픽 : [('소장', 0.15313575340963487), ('게임', 0.1513365759001117), ('센터', 0.14974041425284312), ('연구', 0.11065475553083438), ('실장', 0.09562939690463869), ('교수', 0.09009817975966265), ('주임', 0.08111988301883447), ('병원', 0.07806486468966822), ('과장', 0.07404033060961183), ('의학', 0.07161435412349179)]
2 번째 토픽 : [('지원', 0.03826357929362928), ('건강', 0.034336003432711336), ('정책', 0.034223991949217936), ('코로나', 0.03096849111096097), ('정신', 0.03052066513636486), ('사회', 0.0280120678496208), ('청년', 0.027284064037445828), ('후보', 0.022373150437060455), ('사업', 0.021077478581364228), ('복지', 0.02074661761305505)]
3 번째 토픽 : False
4 번째 토픽 : False
5 번째 토픽 : False
6 번째 토픽 : False
7 번째 토픽 : False
8 번째 토픽 : False
9 번째 토픽 : False


## BERTopic Model - data

In [ ]:
topics, probs = model.fit_transform(docs)

### visualization

In [ ]:
model.visualize_topics()

ValueError: ignored

In [1]:
model.visualize_distribution(probs[0])

NameError: ignored

In [ ]:
for i in range(0, 10):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('매장', 0.3088164924736485), ('벅스', 0.21179397831154362), ('화장실', 0.20407539420616477), ('스타', 0.19977951430179028), ('슐츠', 0.12947550875106217), ('개방', 0.12050744256223027), ('문제', 0.11599772086177139), ('직원', 0.11359686648659695), ('안전', 0.0905217952015817), ('정책', 0.08751614103488498)]
1 번째 토픽 : [('으로', 0.06510150419643676), ('에서', 0.05645409365662946), ('지원', 0.031087110729304233), ('한다', 0.030159158873558356), ('코로나', 0.028742604372185995), ('정신', 0.02870575599069336), ('건강', 0.028617017578185522), ('다고', 0.02766463455063698), ('정책', 0.025225788183307327), ('사회', 0.02490076375448867)]
2 번째 토픽 : False
3 번째 토픽 : False
4 번째 토픽 : False
5 번째 토픽 : False
6 번째 토픽 : False
7 번째 토픽 : False
8 번째 토픽 : False
9 번째 토픽 : False


# blog data topic modeling

## blog data load

In [ ]:
blog_data = pd.read_csv("/content/drive/MyDrive/SKKU_ML/data/blog_data.csv", encoding='utf-8-sig')

In [ ]:
data_list = ['nv', 'cs', 'cs_sum']
for i in data_list:
    with open(f"/content/drive/MyDrive/SKKU_ML/data/blog_{i}.pickle","rb") as fr:
        globals()[f'blog_{i}'] = pickle.load(fr)

In [ ]:
blog_docs = []
for i in range(len(blog_data)):
  blog_docs.append(blog_data.content[i])

## BERTopic - blog_nv

In [ ]:
topics, probs = model.fit_transform(blog_nv)

### visualization

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_distribution(probs[0])

In [ ]:
for i in range(0, 10):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('출처', 0.06195382071508785), ('네이버', 0.023715637754735563), ('메디', 0.02274441841788678), ('뉴스', 0.020910790650396582), ('의료', 0.02052516337683745), ('코로나', 0.019554394776785694), ('신문', 0.018350501857479648), ('병원', 0.0176058577609371), ('데일리', 0.01673909540679943), ('의사', 0.015319300777418198)]
1 번째 토픽 : [('출처', 0.05952940713167715), ('확진', 0.026533166084178175), ('네이버', 0.024479459584257318), ('코로나', 0.02121886081663581), ('뉴스', 0.02040721800199605), ('메디', 0.018200650072690477), ('의료', 0.017990527839438417), ('병원', 0.015488978274611443), ('오미크론', 0.015233579551955614), ('방역', 0.01520310898775896)]
2 번째 토픽 : [('학교', 0.051452179237278954), ('학생', 0.04160963538114835), ('교육', 0.033564577380072916), ('학습', 0.02794177917325718), ('놀이', 0.02371812806611874), ('교사', 0.022871005442349976), ('아이', 0.021391565591173452), ('수업', 0.018344786002834252), ('분쟁', 0.017590484249035518), ('영어', 0.01446893247299246)]
3 번째 토픽 : [('가지', 0.029913099143365908), ('장애', 0.026607944214432737), ('치

# BERTopic - blog_data

In [ ]:
topics, probs = model.fit_transform(blog_docs)

### visualization

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_distribution(probs[0])

In [ ]:
for i in range(0, 10):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('서비스', 0.028335122858657507), ('지원', 0.022146397523554538), ('노인', 0.01977056258629192), ('사회', 0.018537967605383593), ('돌봄', 0.0179452668901349), ('정신', 0.017865831448739336), ('복지', 0.016505070582563484), ('제공', 0.015597011389172263), ('장애', 0.015276728389373875), ('건강', 0.014959281427880106)]
1 번째 토픽 : [('우리', 0.02389631578053673), ('사람', 0.02379844190882562), ('인간', 0.020957992214541303), ('자신', 0.020481441954424865), ('에서', 0.019208708708028707), ('생각', 0.016881539933238644), ('으로', 0.016433023140784497), ('아니', 0.016302804350567256), ('에게', 0.01579826264734419), ('한다', 0.015627240486097923)]
2 번째 토픽 : [('제조', 0.05004457937269979), ('제항', 0.04100422150281755), ('아동', 0.03045131925602495), ('또는', 0.028645055203909815), ('따른', 0.0284952843240204), ('복지', 0.02629984698961686), ('시설', 0.025789646284686097), ('기관', 0.02257838626749994), ('한다', 0.021779511258067534), ('관한', 0.021753280180053843)]
3 번째 토픽 : [('청년', 0.06850514602594443), ('상담', 0.048136282569698485), ('마음', 0.